# Project 4: **Build a Deep Research System**
Welcome to project 4! For this project, we shift our focus from tool use and agents to *reasoning* models. You will practice state‑of‑the‑art inference‑time scaling methods such as *Chain‑of‑Thought* prompting and *Tree‑of‑Thoughts*, and briefly explore high-levels of training reasoning models using techniques like **STaR**.


Finally, you will put everything together to build a *deep research agent* that can browse the web, reason over what it finds, and give structured answers.

## Learning Objectives  
* Apply common inference‑time scaling methods: **zero‑shot / few‑shot CoT, self‑consistency, sequential decoding, tree‑of‑thoughts**  
* Gain intuition for **training** reasoning‑capable models following **STaR** approach 
* Build a minimal **deep‑research agent** that combines step‑by‑step reasoning with live web search   
* Practice extending deep-search to a multi-agent system 

## Roadmap  
1. Environment setup  
2. Inference‑time scaling  
   2.1 Few‑shot & zero‑shot CoT  
   2.2 Self‑consistency
   2.3 Sequential revisions  
   2.4 Tree‑of‑Thought
3. STaR for training models for reasoning  
4. Deep-research agent  
5. (Optional) Multi-agent deep-research

# 1‑ Environment setup

## 1.1- Conda environment

Before we start coding, you need a reproducible setup. Open a terminal in the same directory as this notebook and run:

```bash
# Create and activate the conda environment
conda env create -f environment.yaml && conda activate deep_research

# Register this environment as a Jupyter kernel
python -m ipykernel install --user --name=deep_research --display-name "deep_research"
```
Once this is done, you can select "deep_research" from the Kernel → Change Kernel menu in Jupyter or VS Code.

## 1.2 Ollama setup

In this project we use the `llama3.2:3b` and `deepseek-r1:8b` models. You can try other smaller or larger reasoning LLMs such as `qwen2.5:3b-instruct` or `phi4-mini` to compare performance. Explore available models here: https://ollama.com/library.

```bash
ollama pull llama3.2:3b
ollama pull deepseek-r1:8b
# Additional small reasoning models to compare
# ollama pull qwen2.5:3b-instruct
# ollama pull phi4-mini

```

`ollama pull` downloads the model so you can run it locally without API calls.

---  
# 2‑ Inference‑time scaling

Inference-time scaling refers to techniques that make an existing model reason better without retraining it. Instead of changing the model’s weights, we achieve reasoning capability by adjusting how we prompt, sample, or aggregate LLM's outputs.

In this section, we’ll explore several inference-time strategies that improve reasoning quality using a non-reasoning base model. You will experiment with and compare methods such as:

- Few-shot Chain-of-Thought (CoT)
- Zero-shot CoT
- Self-consistency
- Sequential revision
- Tree-of-Thoughts (ToT)

### 2.1: Few‑Shot CoT
Few-shot prompting helps a model reason by showing one or multiple examples before asking a new question. By observing the pattern of reasoning and final answers, the model learns how to structure its own reasoning process on the new input.

In this exercise, you will create a prompt that includes a few example Q&A pairs demonstrating step-by-step reasoning. Then, you will feed a new question and see the model’s output.

In [1]:
# Step 1: Write a few examples showing reasoning steps
# Step 2: Write your new question
# Step 3: Concatenate examples + new question into a single prompt
# Step 4: Call your Ollama or OpenAI client to get a response from llama3.2:3b # e.g., client.chat.completions.create(...)
# Step 5: Print the final answer

from openai import OpenAI

# Step 1: Reasoning examples
examples = """
Q: If all Bloops are Razzies and all Razzies are Luluz, is every Bloop a Lulu?
A: Yes. Because every Bloop is a Razzie and every Razzie is a Lulu, by transitivity every Bloop is a Lulu.

Q: A bat and a ball cost $1.10 in total. The bat costs $1.00 more than the ball. How much is the ball?
A: Let the ball cost x dollars. Then the bat costs x + 1 dollars. Total: x + (x + 1) = 1.10 → 2x + 1 = 1.10 → 2x = 0.10 → x = 0.05. So the ball costs $0.05.
"""

# Step 2: New question
new_q = "\nQ: If it takes 5 machines 5 minutes to make 5 widgets, how long does it take 100 machines to make 100 widgets?"

# Step 3: Build prompt
prompt = examples + new_q + "\nA:"

# Step 4: Call llama3.2:3b via OpenAI-compatible endpoint
client = OpenAI(base_url="http://localhost:11434/v1", api_key="ollama")
resp = client.chat.completions.create(
    model="llama3.2:3b",
    messages=[{"role": "user", "content": prompt}],
    temperature=0
)

# Step 5: Print answer
print(resp.choices[0].message.content.strip())


To solve this problem, we can use a simple proportionality argument.

If 5 machines can make 5 widgets in 5 minutes, then each machine can make 1 widget in 5 minutes. Therefore, if there are 100 machines, it will take them the same amount of time to make 100 widgets as it took 5 machines to make 5 widgets, which is 5 minutes.

So, the answer is: It takes 100 machines 5 minutes to make 100 widgets.


### (Optional) Few-shot CoT on GPT2
GPT-2 is a pre-trained language model without instruction tuning. It continues text rather than answering questions. In this section, you'll try the exact same CoT pattern on GPT-2 and observe what happens. The goal is to test whether few-shot CoT alone can elicit structured reasoning from a non-chat LLM.

In [3]:
import os
import torch
from transformers import pipeline

# Step 1: Load GPT-2 text-generation from huggingface (https://huggingface.co/docs/transformers/en/model_doc/gpt2)
# Step 2: Write 1–2 few-shot reasoning examples (short, explicit steps + final answer in your own unique format)
# Step 3: Append a new test question after the examples to form one prompt string
# Step 4: Generate 1–3 completions with different decoding settings (e.g., greedy vs. top-k)
# Step 5: Print raw outputs; check if steps are followed and if the final answer is correct

# Step 1: load GPT-2
gen = pipeline("text-generation", model="gpt2", device=0 if torch.cuda.is_available() else -1)

# Step 2: few-shot CoT prompt
prompt = """Q: A bottle and a cork cost $1.10. The bottle is $1 more than the cork. How much is the cork?
Steps: Let cork = x. Then bottle = x + 1. Total: x + (x + 1) = 1.10 → 2x = 0.10 → x = 0.05. Answer: $0.05

Q: If 3 cats catch 3 mice in 3 minutes, how many cats are needed to catch 100 mice in 100 minutes?
Steps: 1 cat catches 1 mouse in 3 min. In 100 min 1 cat catches 100/3 ≈ 33 mice. 100 mice need 100/33 ≈ 3 cats. Answer: 3 cats

Q: A snail climbs 5 ft up a 20 ft wall each day and slips back 4 ft each night. How many days to reach the top?
Steps:"""

# Step 3: generate with different decodings
for kw in [{"do_sample": False}, {"do_sample": True, "top_p": 10, "temperature": 0.8}]:
    out = gen(prompt, max_new_tokens=80, pad_token_id=gen.tokenizer.eos_token_id, **kw)[0]["generated_text"]
    print("-" * 40)
    print(out)

Device set to use cpu


----------------------------------------
Q: A bottle and a cork cost $1.10. The bottle is $1 more than the cork. How much is the cork?
Steps: Let cork = x. Then bottle = x + 1. Total: x + (x + 1) = 1.10 → 2x = 0.10 → x = 0.05. Answer: $0.05

Q: If 3 cats catch 3 mice in 3 minutes, how many cats are needed to catch 100 mice in 100 minutes?
Steps: 1 cat catches 1 mouse in 3 min. In 100 min 1 cat catches 100/3 ≈ 33 mice. 100 mice need 100/33 ≈ 3 cats. Answer: 3 cats

Q: A snail climbs 5 ft up a 20 ft wall each day and slips back 4 ft each night. How many days to reach the top?
Steps: 1 day = 1.5 × 10 = 1.5 × 10 × 10 × 10 × 10 × 10 × 10 × 10 × 10 × 10 × 10 × 10 × 10 × 10 × 10 × 10 × 10 × 10 × 10 × 10 × 10 × 10 × 10 × 10 × 10 × 10 × 10 × 10 × 10 × 10 × 10 × 10 × 10 × 10 × 10
----------------------------------------
Q: A bottle and a cork cost $1.10. The bottle is $1 more than the cork. How much is the cork?
Steps: Let cork = x. Then bottle = x + 1. Total: x + (x + 1) = 1.10 → 2x = 0.10 → x 

### 2.2: Zero‑Shot Chain‑of‑Thought
Zero-shot CoT encourages the model to reason without examples by adding a short cue such as “Let’s think step by step.” This simple phrase often activates the model’s latent reasoning ability even when no demonstrations are provided. It serves as a baseline to compare with few-shot and other inference-time scaling methods.

In [4]:
from openai import OpenAI

# Step 1: Write the question and a zero-shot CoT cue (e.g., "Let's think step by step.")
# Step 2: Build a single prompt string that includes brief role guidance plus the question
# Step 3: Call your Ollama or OpenAI client to get a response from llama3.2:3b  # e.g., client.chat.completions.create(...)
# Step 4: Print the chain and the final answer

# Step 1: zero-shot CoT cue
question = "A lily-pad doubles in size every day. If it covers the whole pond on day 48, on what day does it cover half the pond?"
cue = "Let's think step by step."

# Step 2: build prompt with role guidance
prompt = f"You are a concise math assistant.\nQ: {question}\nA: {cue}"

# Step 3: call llama3.2:3b via Ollama
client = OpenAI(base_url="http://localhost:11434/v1", api_key="ollama")
resp = client.chat.completions.create(
    model="llama3.2:3b",
    messages=[{"role": "user", "content": prompt}],
    temperature=0
)

# Step 4: print chain + answer
print(resp.choices[0].message.content.strip())

To solve this problem, let's analyze the growth pattern of the lily pad.

Since the lily pad doubles in size every day and covers the entire pond on day 48, we can work backwards to find when it covered half the pond.

On day 47, the lily pad must have covered half the pond because it will double in size on day 48 and cover the whole pond.


### 2.3 Self‑Consistency
Self-consistency enhances reasoning accuracy by sampling multiple independent reasoning paths for the same question instead of relying on a single deterministic answer. Each run may follow a slightly different logical chain, and the diversity helps correct individual mistakes. After generating several reasoning traces, you then aggregate the final answers using majority voting.

This approach is especially useful when tasks involve multi-step reasoning or arithmetic, where single-path outputs may be incorrect.

In [5]:
from openai import OpenAI
import re, collections

client = OpenAI(api_key="ollama", base_url="http://localhost:11434/v1")
MODEL = "llama3.2:3b"

def cot_answer(question, temperature=1.0):
    prompt = f"Think step by step.\nQ: {question}\nA:"
    resp = client.chat.completions.create(
        model=MODEL,
        messages=[{"role": "user", "content": prompt}],
        temperature=temperature
    )
    text = resp.choices[0].message.content
    # grab last number as final answer
    nums = re.findall(r"[-+]?\d+(?:\.\d+)?", text)
    return float(nums[-1]) if nums else None

def self_consistent(question, n=10):
    answers = [cot_answer(question, temperature=0.7) for _ in range(n)]
    counter = collections.Counter(a for a in answers if a is not None)
    winner = counter.most_common(1)[0][0]
    return winner, counter

question = "What is the square root of 144?"
winner, counter = self_consistent(question)
print("Votes:", counter)
print("Chosen answer:", winner)

Votes: Counter({12.0: 10})
Chosen answer: 12.0


### 2.4: Sequential Revision

Sequential revision iteratively improves an answer by generating a first draft, critiquing it, and producing revised drafts that condition on prior answers. Each round should be short and focused, so improvements accumulate without drifting from the question.

In [6]:
from openai import OpenAI

client = OpenAI(api_key="ollama", base_url="http://localhost:11434/v1")
MODEL = "llama3.2:3b"

def sequential_revision(question: str, max_steps: int = 3) -> str:
    # Generate an initial draft answer, then iteratively refine it by conditioning each revision on the previous one.
    # Step 1: Ask the model to produce the first draft for the given question
    # Step 2: Loop for max_steps-1 times, each time feeding the last draft back to the model with a request to revise
    # Step 3: Print each draft to observe how the answer evolves
    # Step 4: Return the final improved draft
    draft = client.chat.completions.create(
        model=MODEL,
        messages=[{"role": "user", "content": f"Answer briefly:\nQ: {question}\nA:"}]
    ).choices[0].message.content
    print("Draft 1:", draft)
    for i in range(1, max_steps):
        draft = client.chat.completions.create(
            model=MODEL,
            messages=[{"role": "user", "content": f"Revise and improve this answer:\n{draft}"}]
        ).choices[0].message.content
        print(f"Draft {i+1}:", draft)
    return draft


# Step 1: Define a question that benefits from multi-step reasoning
# Step 2: Call sequential_revision(question, max_steps)
# Step 3: Print the final output
question = "Explain why the sky appears blue."
final = sequential_revision(question, max_steps=3)
print("\nFinal answer:", final)

Draft 1: The sky appears blue because of a phenomenon called Rayleigh scattering, where shorter (blue) wavelengths of light are scattered more than longer (red) wavelengths by the tiny molecules in the Earth's atmosphere.
Draft 2: Here's a revised version:

The sky typically appears blue due to a natural process known as Rayleigh scattering. This phenomenon occurs when sunlight interacts with the tiny molecules present in our atmosphere, such as nitrogen and oxygen gases. As the shorter-wavelength light (blue) rays pass through these molecules, they are scattered more extensively than longer-wavelength light (red). Consequently, our eyes perceive the scattered blue light more intensely than the remaining red light, resulting in the characteristic blue color of the sky.

I made several revisions to improve the answer:

1. Started with a more concise statement ("The sky typically appears blue") and provided context for why this is so.
2. Introduced the Rayleigh scattering process in a mo

### 2.5 Tree‑of‑Thoughts
Tree-of-Thoughts reframes reasoning as a search process rather than a single forward chain.
Instead of producing one linear sequence of thoughts, the model generates multiple candidate thoughts at each step, evaluates their promise, and then expands only the best few. This allows exploration of different reasoning paths before committing to a final answer, similar to how humans brainstorm, prune, and refine ideas.


In this section, you’ll experiment with two simplified versions of ToT:
1. Word Ladder puzzle solver: a small example where each “thought” is a candidate word transition.
2. Generic ToT search (depth 2, width 2): a minimal logic to expand, evaluate, and select reasoning branches

In [10]:
###### Word Ladder Puzzle ##########

def neighbors(word, vocabulary):
    # Generate all valid one-letter mutations of 'word' that exist in 'vocabulary' and return them.
    # all one-letter mutations that stay in vocab
    w = list(word)
    out = []
    for i in range(len(w)):
        for c in 'abcdefghijklmnopqrstuvwxyz':
            old = w[i]
            w[i] = c
            new = ''.join(w)
            if new in vocabulary and new != word:
                out.append(new)
            w[i] = old
    return out


def tree_of_thought(start, goal, vocab, max_depth=5, beam_width=3):
    # Search over partial thoughts (paths) using a small beam.
    # Step 1: Initialize the frontier with a single path [start]
    # Step 2: For each depth, expand each path by one neighbor from 'neighbors'
    # Step 3: Score paths by edit distance between last word and 'goal' (smaller is better)
    # Step 4: Keep the top 'beam_width' paths and stop early if any reaches 'goal'
    # Step 5: Return the best goal-reaching path or None
    from collections import deque
    import editdistance
    frontier = deque([[start]])          # beam of paths
    for _ in range(max_depth):
        candidates = []
        for path in frontier:
            for nxt in neighbors(path[-1], vocab):
                if nxt in path:          # no cycles
                    continue
                new_path = path + [nxt]
                if nxt == goal:
                    return new_path
                score = editdistance.eval(nxt, goal)
                candidates.append((score, new_path))
        candidates.sort(key=lambda x: x[0])
        frontier = deque([p for _, p in candidates[:beam_width]])
        if not frontier:
            break
    return None


vocab = {"hit","dot","cog","log","dog","lot","lit","hot"}
print(tree_of_thought("hit", "cog", vocab)) # one candidate solution: ['hit', 'hot', 'dot', 'dog', 'cog']


['hit', 'hot', 'dot', 'dog', 'cog']


In [11]:
###### Generic ToT Search ##########

import re
from openai import OpenAI
client = OpenAI(api_key="ollama", base_url="http://localhost:11434/v1")
MODEL = "llama3.2:3b"

def propose_thoughts(question, state, k=2):
    # Propose up to k next “thoughts” that extend the current partial solution/state.
    # Steps: build a short prompt with problem + current state; call your client with n=k. Then return a list of stripped strings (≤ k).
    prompt = f"Problem: {question}\nCurrent plan: {state}\nGive {k} brief next ideas (one per line):"
    resp = client.chat.completions.create(
        model=MODEL,
        messages=[{"role": "user", "content": prompt}],
        n=k, temperature=0.7
    )
    return [choice.message.content.strip() for choice in resp.choices[:k]]


def score_state(question, state):
    # Score how promising a partial solution is on a 1–10 scale (higher is better).
    # Steps: build a rating prompt; call the model; parse the first integer 1–10;
    prompt = f"Problem: {question}\nPlan: {state}\nRate this plan 1-10:"
    txt = client.chat.completions.create(
        model=MODEL,
        messages=[{"role": "user", "content": prompt}],
        temperature=0
    ).choices[0].message.content
    m = re.search(r"\b([1-9]|10)\b", txt)
    return int(m.group(1)) if m else 5


def tree_of_thoughts(question, depth=2, width=2):
    # Run a tiny ToT search: expand states with propose_thoughts, score with score_state, keep top-k at each depth.
    # Steps: initialize frontier=[("", 0)]; for each depth, expand each state with k=width thoughts; score each; sort by score desc; keep top 'width'; return best state and score.
    frontier = [("", 0)]          # (state, score)
    for _ in range(depth):
        extended = []
        for state, _ in frontier:
            for thought in propose_thoughts(question, state, k=width):
                new_state = f"{state} {thought}".strip()
                scr = score_state(question, new_state)
                extended.append((new_state, scr))
        extended.sort(key=lambda x: -x[1])
        frontier = extended[:width]
    best_state, best_score = frontier[0]
    return best_state, best_score


question = "Design a plan for a weekend science workshop for 12-year-olds."
solution, score = tree_of_thoughts(question)

print(f"Best solution (score {score}):\n{solution}")

Best solution (score 1):
It seems you're giving me the current plan, which is not an actual plan. Could you please provide more context or details about the current plan? I'd be happy to help design a new plan for a weekend science workshop for 12-year-olds.

If you don't have a plan yet, here's a general outline:

**Weekend Science Workshop Plan**

**Age Group:** 12-year-olds

**Duration:** Saturday and Sunday (8am-5pm)

**Objective:** To provide hands-on science experiences that foster curiosity, creativity, and critical thinking in young minds.

Here are some potential ideas to get you started:

* **Theme:** "Exploring the Natural World" or "The Science of Wonder"
* **Activities:**
 + Hands-on experiments with everyday materials (e.g., slime-making, baking soda volcanoes)
 + Guided nature walks or outdoor explorations
 + STEM challenges and puzzles
 + Guest speakers from related fields (e.g., biology, chemistry, physics)

Let me know if you'd like to elaborate on any of these ideas 

---  
# 3‑ Training Models for Reasoning

### 3.1: CoT Training
Chain-of-Thought (CoT) training conditions the model on explicit rationales during fine-tuning. Instead of teaching the model to output only the final answer, we train on (question, rationale, answer) so the model learns to internalize multi-step reasoning patterns. A practical recipe is STaR (Self-Taught Reasoner), which uses a stronger teacher model to bootstrap rationales that a smaller student can learn from.

For tasks that require multi-hop reasoning, models fine-tuned on rationales often achieve higher accuracy and are more stable at inference time than models trained on direct answers only. 

Training a full language model is beyond the scope of this notebook, but here is the high-level workflow followed by a short pseudocode:
- Collect questions: Prepare a dataset of questions and correct answers.
- Generate rationales: Use a strong LLM to produce step-by-step reasoning ending with the correct answer.
- Filter and clean: Discard incorrect or low-quality rationales.
- Prepare training data: Format triples (question, rationale, answer) for supervised fine-tuning.
- Fine-tune: Fine-tune the LLM on rationales.
- Iterate: Refine prompts, improve data quality, and retrain for stronger reasoning.

In [ ]:
# Pseudocode (STaR loop)
# for round in 1 ... iters:
    # STEP 1: self-generate reasoning (teacher creates rationale + answer)
    # STEP 2: keep only correct, high-quality traces
    # STEP 3: fine-tune student on (question, rationale, answer) data

### 3.2: ORM vs PRM + RL
Training a Reward Model (RM) allows large language models to be improved through reinforcement learning (RL). Instead of fine-tuning directly on examples, we train a separate model that can score or rank model outputs, and use those scores as feedback signals to refine the policy model.

Two main reward modeling approaches are ORM (predicts a scalar reward for the final answer) and PRM (evaluates the reasoning steps instead of just the outcome)



| Approach | Typical loss | When to use |
|-----------|-------------|-------------|
|*Outcome Reward Model* | Predict scalar reward | Easy to collect training data using verifiers |
|*Process Reward Model* | Predict rewards per step | Difficult to collect training data but more accurate |
| *RLHF* | Use RM as reward in **RL** fine‑tuning | Aligns policy with human signals | Aligns model policy with human or synthetic preferences




In [ ]:
# for round = 1 ... iters:
    # STEP 1:  Generate reasoning
        # sample a minibatch of questions
        # policy roll‑out (actions + log‑probs)
    # STEP 2:  Score the trajectory
        # ORM: scalar reward for the final answer / PRM: scalar reward for the thought process
    # STEP 3:  Reinforce the policy (PPO)

---  
# 4‑ A Deep Research Agent

A deep-research agent pairs a reasoning model (e.g., deepseek-r1) with external tools for web search and retrieval. We will follow the ReAct pattern: the model writes short thoughts, decides when to call tools, reads observations, and continues reasoning until it can answer or reaches a step limit.

We now combine a **search tool** with a reasoning model (e.g., `deepseek-r1`) in a multi-step setup. We follow the *ReAct* pattern (reason → tool → observation):

1. The model reasoins and decides to use tools
2. The agent searches and feed condensed snippets back as context
3. Iterate until the model answers or hits a step limit

We use `AgentType.OPENAI_FUNCTIONS`, which hides the loop inside the LangChain agent.

In [12]:
from ddgs import DDGS
from langchain.tools import Tool

def ddg_search(query: str, k: int = 5) -> str:
    # Use DDGS to run a simple web search and return joined snippets.
    with DDGS() as ddgs:
        results = ddgs.text(query, max_results=k)
        snippets = [r["body"] for r in results if "body" in r]
    return "\n".join(snippets) if snippets else "No results."

search_tool = Tool(
    name="DuckDuckGo Search",
    func=ddg_search,
    description="Search the public web. Input: a plain English query. Returns: concatenated snippets."
)


In [ ]:
from langchain.agents import initialize_agent, AgentType
from langchain_community.chat_models import ChatOllama

MODEL = "deepseek-r1:8b"
question = "What are the best resources to learn machine learning in 2025?"

# Step 1: Initialize the reasoning model via ChatOllama
llm = ChatOllama(model=MODEL)

# Step 2: Build the agent with tool access (DuckDuckGo Search) and function-calling interface (initialize_agent)
agent = initialize_agent(
    tools=[search_tool],
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)


# Step 3: Ask a query and let the agent search + reason to produce an answer
response = agent.run(question)
print(response)

# Optional (Multi-agent Deep Research)
Instead of a single multi-step agent, you can design multiple collaborating agents such as a Planner, Searcher, Summarizer, and Verifier that pass information and refine each other’s outputs. This setup improves robustness, diversity of reasoning, and division of labor.

Try building a simple setup with 2–3 agents that share goals and messages, for example Planner → Researcher → Writer.

In [ ]:
def parallel_research(query, n=3):
    # Run n independent research runs in parallel and return their answers.
    # Steps: use ThreadPoolExecutor; submit n calls to your agent/search pipeline; gather results in order.
    """
    YOUR CODE HERE
    """

answers = parallel_research("What are the best resources to learn ML in 2025?")
for i,a in enumerate(answers,1):
    print(f"[Run {i}] {a[:200]}…")

## 🎉 Congratulations!

* Practised various inference‑time reasoning methods
* Gained intuition about training reasoning models
* You have built a **deep-research agent**: reasoning model like deep-seek r1 + ReAct-style agent + tool use (web search)
* Try adding more tools, and extending the deep-research to a multi-agent system: many agents researching web in parallel.


👏 **Great job!** Take a moment to celebrate. The techniques you implemented here power many production agents and chatbots.